In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
from pypfopt import risk_models, EfficientFrontier
from pypfopt import black_litterman

In [ ]:
precios_df = pd.read_csv('water_assets.csv', index_col='Fecha', parse_dates=True)
capitalizacion_semanal = pd.read_csv('capitalizacion_semanal.csv', index_col='Semana', parse_dates=True)
errores = pd.read_csv('errores.csv', index_col='Activo')
esg_f = pd.read_csv('esg_score.csv', index_col='Activo')
views = pd.read_csv('predicciones_df.csv', index_col='Fecha', parse_dates=True)
tipo_interes_df = pd.read_csv('tipo_interes.csv', index_col='Fecha', parse_dates=True)

In [ ]:
# Convertir la columna 'ESG Score' a tipo numérico
esg_f['ESG Score'] = pd.to_numeric(esg_f['ESG Score'])
# Crear un diccionario a partir del DataFrame esg_f
esg_dict = esg_f.set_index('asset')['ESG Score'].to_dict()

In [ ]:
precios_df = precios_df.fillna(method='bfill')

In [ ]:
precios_medios=precios_m2.mean()
errores_p=(1-errores/precios_medios)/100
errores_p=errores_p[errores_p>0].fillna(0).values
omega=np.zeros((56,56))
np.fill_diagonal(omega, errores_p)

In [ ]:
# Parámetros del modelo
tau = 0.25  # Escalar de incertidumbre
omega = None  # Si es None, se calculará automáticamente basado en tau y cov_matrix
results_weights = []  
results_rets = []   
# Iteración semanal
for i in range(capitalizacion_semanal.shape[0]):
    # Datos históricos ajustados para cada iteración
    precios_d = precios_df.iloc[:precios_df.shape[0] - 53 + i]
    cap = capitalizacion_semanal.iloc[i]
    mcaps = cap.to_dict()

    # Calcular matriz de covarianza (sampleada y ajustada)
    prior_cov_matrix = risk_models.sample_cov(precios_d)

    # Calcular delta (aversión al riesgo) a partir de precios históricos
    rf = tipo_interes_semanal.iloc[i] 
    delta = black_litterman.market_implied_risk_aversion(precios_df_backfilled, risk_free_rate=rf)
    viewdict = views.iloc[i].to_numpy()
    # Configuración de Black-Litterman con vistas absolutas
    bl = BlackLittermanModel(
        prior_cov_matrix,
        absolute_views=viewdict,
        market_caps=mcaps,
        risk_aversion=delta,
        tau=tau,
        omega=omega
    )
    # Obtener retornos y covarianza posterior
    posterior_rets = bl.bl_returns()  # Retornos posteriores ajustados
    posterior_cov_matrix = bl.bl_cov()  # Covarianza posterior ajustada

    
    ef = EfficientFrontier(posterior_rets, posterior_cov_matrix, weight_bounds=(0, 0.1))
     # Añadir restricción ESG
    ef.add_constraint(lambda weights: cp.sum(cp.multiply(weights, list(esg_dict.values()))) >= 3)
    # Optimización en la frontera eficiente con restricción de varianza
    weights = ef.efficient_risk(target_volatility=0.25)  # Varianza máxima permitida (0.25)
    clean_weights = ef.clean_weights()

    # Guardar resultados
    results_weights.append(clean_weights)
    results_rets.append(posterior_rets)

# Construcción de DataFrames con resultados
results_weights_df = pd.DataFrame(results_weights)
results_rets_df = pd.DataFrame(results_rets)